In [ ]:
import os, sys
from pathlib import Path
sys.path.insert(0, Path(os.getcwd()) / 'services/qure_platform_api/hct_api/hct_api')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings.py")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()
import warnings
warnings.filterwarnings('ignore')


from image_manager.models import *
from portal_manager.models import *
from result_manager.models import *
from portal_manager.models.user_activity import UserActivityTracker
from job_manager.models import *
from model_manager.models import *
from qure_keycloak_auth.models import *
from model_manager.ct_tasks.tasks import general as general_tasks
from qure_keycloak_auth.models.account import OrganizationSource
from model_manager.qer_tasks import scheduler as qer_scheduler
from model_manager.tasks import submit_computation_tasks
from model_manager.common import cleanup
from model_manager.views import ProcessSeries
import pprint
from datetime import datetime, timedelta
from django.contrib.auth.models import User
from model_manager.qxr_tasks import scheduler as qxr_scheduler

def get_aws_path(series_id, bucket='in-qureapp'):
    i = ImageSeries.objects.get(id=series_id)
    return 'aws s3 sync s3://{}/{}/{} {}'.format(bucket,i.sourceID,i.seriesInstanceUID,i.seriesInstanceUID)

def delete_graph(series_id):
    try:
        g = Graph.all_objects.get(reference_uuid="series_{}".format(series_id))
        g.interrupt(),g.delete(),print("graph deleted")
    except:print("graph does not exist")
def reprocess(series_id,priority=-9):
    delete_graph(series_id)
    series = ImageSeries.objects.get(id=series_id)
    try:
        if series.usecase=='qxr':
            qxr_scheduler.submit_validation_tasks(series.instances.first().id)
            series.mark_as_pushed()
        else:
            graph, created = series.get_or_create_compute_graph()
            Task.create_task(graph=graph,priority=priority,fn=submit_computation_tasks,arguments={"series_id": series.id})
    except Exception as e:print(e)

def get_tasks(series_id):
    ta=[]
    for t in Task.all_objects.filter(graph__reference_uuid='series_{}'.format(series_id)).order_by('id'):
        ta.append([t.id,t.type.split(".")[-1],t.status])
    return ta

def add_to_all(email='paras.jain@qure.ai'):
    a = Account.objects.get(email=email)
    for o in OrganizationSource.objects.all():
        a.sources.add(o)
        a.save()
def last_failed_task(series_id):
    all_tasks=get_tasks(series_id)
    for t in all_tasks:
#         print(t[2])
        if t[2] != 2:
            task = Task.all_objects.get(id=t[0])
            return task.type.split(".")[-1],task.results, task.status
    
def change_priority(series):
    for t in Task.objects.filter(graph__reference_uuid='series_{}'.format(series.id)).order_by('id'):
        t.priority=-10
        t.save()
    print("priority changed")